# Punto 1 (Exploración técnicas de Clustering)

## Introducción al Clustering y K-Means

En el presente notebook se exploran las técnicas de clustering, se implementará el algoritmo K-means desde cero sin utilizar librerías especializadas para clustering (pero sí para manejo de datos y visualización). Se utilizarán **datos sintéticos** con **5 clusters** en **2D y 3D**, cuyas **etiquetas verdaderas** son conocidas. Esto permitirá **comparar los resultados del algoritmo** con las etiquetas originales.

### ¿Qué es el Clustering?
El **Clustering** es una técnica de **aprendizaje no supervisado** que consiste en **agrupar datos** en **conjuntos** o **clusters** basándose en su **similitud**.  
El objetivo es que los **puntos dentro de un mismo cluster** sean **lo más similares posible** entre sí, mientras que los **puntos en clusters diferentes** sean **lo más diferentes posible**.  

El clustering se utiliza en diversos campos, como:
- **Análisis de clientes**: Para agrupar clientes con comportamientos de compra similares.  
- **Segmentación de imágenes**: Para identificar regiones similares en una imagen.  
- **Agrupación de documentos**: Para organizar grandes volúmenes de texto en temas o categorías.

---

### ¿Qué es K-Means?
**K-Means** es uno de los algoritmos de **clustering** más **populares** y **simples**.  
Funciona agrupando los datos en **K clusters**, donde **K** es un número definido previamente.  
El objetivo de K-Means es **minimizar la varianza intra-cluster** y **maximizar la separación inter-cluster**.

#### **¿Cómo funciona?**  
1. **Inicialización**: Se eligen aleatoriamente **K centroides** (uno por cada cluster).  
2. **Asignación de puntos**: Cada punto de datos se asigna al **centroide más cercano**.  
3. **Recalculación de centroides**: Se **recalcula la posición** de cada centroide como la **media** de los puntos asignados a su cluster.  
4. **Repetición**: Los pasos 2 y 3 se **repiten hasta converger**, es decir, hasta que se supere la cantidad de iteraciones máximas o que ya no exista un cambio significitativo entre el cálculo de los centroides.

### **Características de K-Means**:  
- Es **rápido y eficiente** en datasets grandes.  
- Se requiere **especificar el número de clusters (K)** de antemano.  
- Utiliza una **métrica de distancia** (usualmente **Euclidiana**) para asignar puntos a los clusters.  
- Puede **converger a mínimos locales**, dependiendo de la **inicialización de los centroides**.

## Importación librerías
En este proyecto se hará uso de las librerías indicadas en el documento _requirements.txt_ para la gestión de datos y visualización. A continuación se importan las librerías necesarias (pandas, numpy, matplotlib, scipy, entre otras).

In [ ]:
import numpy as np
import pandas as pd

## Lectura de datos
Se hará la lectura de dos datasets de datos sintéticos, uno en 2D y otro en 3D (_data_2d_ y _data_3d_ respectivamente).

In [ ]:
# Lectura de datos 2d
data_2d = pd.read_csv('data_2d.csv')
data_2d.head()

# Lectura de datos 3d
data_3d = pd.read_csv('data_3d.csv')
data_3d.head()